In [3]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Bidirectional
from keras.initializers import Constant
from sklearn.model_selection import train_test_split
import numpy as np

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
df = pd.read_csv('/content/IMDB Dataset.csv')



In [7]:
df.head()


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [8]:
df.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [9]:
# Assuming the reviews are in a column named 'review' and sentiments in a column named 'sentiment'
review = df['review'].tolist()
sentiment = df['sentiment'].tolist()

# Now, 'reviews' and 'sentiments' are lists containing the review texts and sentiment labels, respectively.


In [10]:
import numpy as np

def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word]
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

In [11]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing. sequence import pad_sequences
import numpy as np

maxlen = 100 # We will cut reviews after 100 words
training_samples = 40000 # We will be training on 40000 samples
max_words = 10000 # We will only consider the top 10,000 words in the dataset
testing_samples=10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(review)
sequences = tokenizer. texts_to_sequences (review)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)

sentiment = np.asarray(sentiment)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', sentiment. shape)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
sentiment = sentiment[indices]

x_train = data[:training_samples]
y_train = sentiment[:training_samples]

x_test=data[testing_samples:]
y_test=sentiment[testing_samples:]

Found 124252 unique tokens.
Shape of data tensor: (50000, 100)
Shape of label tensor: (50000,)


In [12]:
y_test

array(['positive', 'positive', 'positive', ..., 'negative', 'positive',
       'negative'], dtype='<U8')

In [13]:
y_val

array(['positive', 'positive', 'negative', ..., 'negative', 'positive',
       'negative'], dtype='<U8')

In [14]:
embeddings_index = {}

# Open the GloVe file
with open('/content/glove.6B.100d.txt', encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print(f'Found {len(embeddings_index)} word vectors.')


Found 287203 word vectors.


In [15]:
embeddings_index['bad']

array([ 0.39456  , -0.24717  ,  1.0319   , -0.61444  , -1.2376   ,
        0.051794 , -0.55112  , -0.31014  , -0.025735 , -0.24709  ,
        0.25836  ,  0.34899  ,  0.46176  ,  0.44237  ,  0.354    ,
        0.21262  , -0.26312  , -0.17128  ,  0.44013  ,  1.1284   ,
        0.37259  ,  0.41969  , -0.2534   , -0.051885 , -0.34917  ,
        0.15298  , -0.68444  , -0.19514  , -0.018647 ,  0.084191 ,
        0.14569  ,  0.3869   ,  0.023524 , -0.37861  , -0.65081  ,
        0.010398 , -0.50087  , -0.087368 ,  0.098617 , -0.21907  ,
       -0.072412 , -0.10247  , -0.3148   , -0.307    , -0.30457  ,
       -0.14329  ,  0.10313  , -0.040487 , -0.095922 , -1.192    ,
       -0.063356 ,  0.034127 , -0.057674 ,  0.56582  ,  0.082541 ,
       -2.4617   ,  0.48029  ,  0.72806  ,  1.2841   ,  0.10672  ,
       -0.045119 ,  1.0117   , -0.97928  ,  0.022828 ,  0.46876  ,
       -0.026875 ,  0.80018  , -0.23101  ,  0.086699 , -0.74338  ,
       -0.38738  , -0.1924   , -0.13292  ,  0.52347  ,  0.3565

In [16]:
embedding_dim = 100
embedding_matrix = np.zeros ((max_words, embedding_dim))
for word, i in word_index.items():
  embedding_vector = embeddings_index.get(word)
  if i < max_words:
    if embedding_vector is not None:
      # Words not found in embedding index will be all-zeros.
      embedding_matrix[i] = embedding_vector

In [17]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
model = Sequential()


model.add(Embedding (max_words, embedding_dim, input_length=maxlen))
model.add(Dense(10,activation="relu"))
model.add(Flatten())

model.add(Dense(1,activation="sigmoid"))

model.summary()






Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          1000000   
                                                                 
 dense (Dense)               (None, 100, 10)           1010      
                                                                 
 flatten (Flatten)           (None, 1000)              0         
                                                                 
 dense_1 (Dense)             (None, 1)                 1001      
                                                                 
Total params: 1002011 (3.82 MB)
Trainable params: 1002011 (3.82 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
model.layers[0].set_weights ([embedding_matrix])
model.layers[0].trainable = False


In [20]:
#convert y_train and y_val to binary
from sklearn.preprocessing import LabelEncoder

# Example labels (replace these with your actual y_train and y_val)


# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform labels to binary format
y_train_binary = label_encoder.fit_transform(y_train)
y_val_binary = label_encoder.transform(y_val)
y_test_binary=label_encoder.transform(y_test)



# Display the transformed labels
print("Transformed y_train:", y_train_binary)
print("Transformed y_val:", y_val_binary)
print("Transformed y_val:", y_test_binary)


#saving the y_train and y_val if we see a problem
y_train_og=y_train
y_val_og=y_val
y_test_og=y_test

#changing y_val and y_train to 0s and 1s

y_train=y_train_binary
y_val=y_val_binary
y_test=y_test_binary

Transformed y_train: [1 0 1 ... 0 0 1]
Transformed y_val: [1 1 0 ... 0 1 0]
Transformed y_val: [1 1 1 ... 0 1 0]


In [21]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
model.fit(x_train, y_train, epochs=20, batch_size=256)


Epoch 1/20
157/157 [==============================] - 4s 18ms/step - loss: 0.6539 - acc: 0.6096
Epoch 2/20
157/157 [==============================] - 2s 12ms/step - loss: 0.5288 - acc: 0.7442
Epoch 3/20
157/157 [==============================] - 2s 12ms/step - loss: 0.4726 - acc: 0.7787
Epoch 4/20
157/157 [==============================] - 2s 12ms/step - loss: 0.4490 - acc: 0.7921
Epoch 5/20
157/157 [==============================] - 2s 12ms/step - loss: 0.4340 - acc: 0.7990
Epoch 6/20
157/157 [==============================] - 2s 11ms/step - loss: 0.4219 - acc: 0.8059
Epoch 7/20
157/157 [==============================] - 2s 16ms/step - loss: 0.4133 - acc: 0.8092
Epoch 8/20
157/157 [==============================] - 3s 19ms/step - loss: 0.4060 - acc: 0.8136
Epoch 9/20
157/157 [==============================] - 2s 13ms/step - loss: 0.4007 - acc: 0.8180
Epoch 10/20
157/157 [==============================] - 2s 11ms/step - loss: 0.3940 - acc: 0.8206
Epoch 11/20
157/157 [==================

In [22]:
#Unidirectional LSTM layer

# Define parameter
n_lstm = 128


model1 = Sequential()


model1.add(Embedding (max_words, embedding_dim, input_length=maxlen))
model1.add(LSTM(n_lstm, return_sequences=False))
model1.add(Flatten())

model1.add(Dense(1,activation="sigmoid"))

model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 100)          1000000   
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 flatten_1 (Flatten)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1117377 (4.26 MB)
Trainable params: 1117377 (4.26 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
model1.layers[0].set_weights ([embedding_matrix])
model1.layers[0].trainable = False


In [24]:
model1.compile(optimizer='adam',loss='binary_crossentropy', metrics=['acc'])
model1.fit(x_train, y_train, epochs=20, batch_size=256)
# model1.save_weights ('pre_trained_glove_model.h5')

Epoch 1/20
157/157 [==============================] - 78s 485ms/step - loss: 0.5493 - acc: 0.7155
Epoch 2/20
157/157 [==============================] - 74s 474ms/step - loss: 0.4548 - acc: 0.7851
Epoch 3/20
157/157 [==============================] - 73s 468ms/step - loss: 0.4090 - acc: 0.8138
Epoch 4/20
157/157 [==============================] - 72s 460ms/step - loss: 0.3782 - acc: 0.8281
Epoch 5/20
157/157 [==============================] - 74s 473ms/step - loss: 0.3537 - acc: 0.8420
Epoch 6/20
157/157 [==============================] - 74s 475ms/step - loss: 0.3327 - acc: 0.8531
Epoch 7/20
157/157 [==============================] - 73s 463ms/step - loss: 0.3153 - acc: 0.8617
Epoch 8/20
157/157 [==============================] - 77s 489ms/step - loss: 0.3009 - acc: 0.8693
Epoch 9/20
157/157 [==============================] - 74s 475ms/step - loss: 0.2892 - acc: 0.8753
Epoch 10/20
157/157 [==============================] - 77s 489ms/step - loss: 0.2749 - acc: 0.8825
Epoch 11/20
157/157

In [25]:
#Bidirectional LSTM layer(Incomplete)

# Define parameter
n_lstm = 128


model2 = Sequential()


model2.add(Embedding (max_words, embedding_dim, input_length=maxlen))
model2.add(Bidirectional(LSTM(64,return_sequences = False)))
model2.add(Flatten())
model2.add(Dense(1,activation="sigmoid"))


In [31]:
model2.layers[0].set_weights ([embedding_matrix])
model2.layers[0].trainable = False


In [32]:
model2.compile(optimizer='adam',loss='binary_crossentropy', metrics=['acc'])
model2.fit(x_train, y_train, epochs=20, batch_size=256)
# model1.save_weights ('pre_trained_glove_model.h5')


Epoch 1/20
157/157 [==============================] - 76s 450ms/step - loss: 0.9110 - acc: 0.5634
Epoch 2/20
157/157 [==============================] - 69s 442ms/step - loss: 0.5268 - acc: 0.7431
Epoch 3/20
157/157 [==============================] - 71s 452ms/step - loss: 0.4460 - acc: 0.7935
Epoch 4/20
157/157 [==============================] - 75s 474ms/step - loss: 0.4135 - acc: 0.8113
Epoch 5/20
157/157 [==============================] - 69s 440ms/step - loss: 0.3967 - acc: 0.8208
Epoch 6/20
157/157 [==============================] - 72s 457ms/step - loss: 0.3770 - acc: 0.8317
Epoch 7/20
157/157 [==============================] - 69s 440ms/step - loss: 0.3669 - acc: 0.8340
Epoch 8/20
157/157 [==============================] - 75s 478ms/step - loss: 0.3517 - acc: 0.8435
Epoch 9/20
157/157 [==============================] - 69s 440ms/step - loss: 0.3462 - acc: 0.8460
Epoch 10/20
157/157 [==============================] - 71s 455ms/step - loss: 0.3366 - acc: 0.8515
Epoch 11/20
157/157

In [28]:
train_dense_results = model.evaluate(x_train, y_train, verbose=2, batch_size=256)
valid_dense_results = model.evaluate(x_test, y_test, verbose=2, batch_size=256)
print(f'Train accuracy: {train_dense_results[1]*100:0.2f}')
print(f'Test accuracy: {valid_dense_results[1]*100:0.2f}')

157/157 - 2s - loss: 0.3750 - acc: 0.8332 - 2s/epoch - 11ms/step
157/157 - 1s - loss: 0.3853 - acc: 0.8270 - 966ms/epoch - 6ms/step
Train accuracy: 83.32
Test accuracy: 82.70


In [29]:
train_dense_results = model1.evaluate(x_train, y_train, verbose=2, batch_size=256)
valid_dense_results = model1.evaluate(x_test, y_test, verbose=2, batch_size=256)
print(f'Train accuracy: {train_dense_results[1]*100:0.2f}')
print(f'Test accuracy: {valid_dense_results[1]*100:0.2f}')

157/157 - 33s - loss: 0.0999 - acc: 0.9660 - 33s/epoch - 213ms/step
157/157 - 34s - loss: 0.1814 - acc: 0.9377 - 34s/epoch - 214ms/step
Train accuracy: 96.60
Test accuracy: 93.77


In [33]:
train_dense_results = model2.evaluate(x_train, y_train, verbose=2, batch_size=256)
valid_dense_results = model2.evaluate(x_test, y_test, verbose=2, batch_size=256)
print(f'Train accuracy: {train_dense_results[1]*100:0.2f}')
print(f'Test accuracy: {valid_dense_results[1]*100:0.2f}')

157/157 - 32s - loss: 0.2367 - acc: 0.9009 - 32s/epoch - 201ms/step
157/157 - 31s - loss: 0.2626 - acc: 0.8890 - 31s/epoch - 197ms/step
Train accuracy: 90.09
Test accuracy: 88.90
